In [13]:
import sys

sys.path.append("lib")

from functorch import vmap

import matplotlib.pyplot as plt

import lib
import torch
import math
import numpy

In [14]:
device = torch.device("cpu")

In [15]:
Nx = 100
Ny = 100
mesh = lib.mesh.RectangleMesh({"xmin": 0, "xmax": 1, "ymin": 0, "ymax": 1, "Nx": Nx, "Ny": Ny}, device)
left = torch.isclose(mesh.nodes[:,0], torch.zeros(1, device=device)).nonzero().squeeze(1)
right = torch.isclose(mesh.nodes[:,0], torch.ones(1, device=device)).nonzero().squeeze(1)
bottom = torch.isclose(mesh.nodes[:,1], torch.zeros(1, device=device)).nonzero().squeeze(1)
top = torch.isclose(mesh.nodes[:,1], torch.ones(1, device=device)).nonzero().squeeze(1)

In [16]:
problem = lib.problem.FEProblem(mesh)
problem.add_variable("ux", lambda x : torch.rand((1,), device=device))
# problem.add_variable("uy", lambda x : torch.sin(x[1]))
problem.add_weakform(lib.weakform.Diffusion(problem, {"variable":"ux", "diffusivity":1e-5}))
# problem.add_weakform(lib.weakform.Diffusion(problem, {"variable":"uy"}))
problem.add_weakform(lib.weakform.TimeDirivative(problem, {"variable":"ux"}))
problem.init_solution()
problem.write_step("foo.vtk", 0)

In [17]:
# problem.add_nodal_bc(lib.bc.DirichletBC(problem, {"value":0,"boundary":left,"variable":"ux"}))
# problem.add_nodal_bc(lib.bc.DirichletBC(problem, {"value":0.1,"boundary":right,"variable":"ux"}))
# problem.add_nodal_bc(lib.bc.DirichletBC(problem, {"value":-0.1,"boundary":bottom,"variable":"uy"}))
# problem.add_nodal_bc(lib.bc.DirichletBC(problem, {"value":0.1,"boundary":top,"variable":"uy"}))

In [18]:
for i in range(100):
    problem.advance_in_time(0.01)
    problem.print_time()
    problem.solve()
    problem.write_step("foo.vtk", i + 1)

time = 1.000000E-02, dt = 1.000000E-02
Iteration 0, ||r|| = 8.169E-04
Iteration 1, ||r|| = 2.853E-17
time = 2.000000E-02, dt = 1.000000E-02
Iteration 0, ||r|| = 8.082E-04
Iteration 1, ||r|| = 2.809E-17
time = 3.000000E-02, dt = 1.000000E-02
Iteration 0, ||r|| = 7.996E-04
Iteration 1, ||r|| = 2.756E-17
time = 4.000000E-02, dt = 1.000000E-02
Iteration 0, ||r|| = 7.912E-04
Iteration 1, ||r|| = 2.806E-17
time = 5.000000E-02, dt = 1.000000E-02
Iteration 0, ||r|| = 7.829E-04
Iteration 1, ||r|| = 2.806E-17
time = 6.000000E-02, dt = 1.000000E-02
Iteration 0, ||r|| = 7.747E-04
Iteration 1, ||r|| = 2.812E-17
time = 7.000000E-02, dt = 1.000000E-02
Iteration 0, ||r|| = 7.666E-04
Iteration 1, ||r|| = 2.778E-17
time = 8.000000E-02, dt = 1.000000E-02
Iteration 0, ||r|| = 7.586E-04
Iteration 1, ||r|| = 2.830E-17
time = 9.000000E-02, dt = 1.000000E-02
Iteration 0, ||r|| = 7.508E-04
Iteration 1, ||r|| = 2.773E-17
time = 1.000000E-01, dt = 1.000000E-02
Iteration 0, ||r|| = 7.430E-04
Iteration 1, ||r|| = 